# transformer
attention 由由bengio团队于2014年提出， 并在近年来得到广泛的应用。
transformer中抛弃了传统

In [1]:
# t 是

In [6]:
import numpy as np
import tensorflow as tf

In [3]:
def get_angles(pos, i, d_model):
    # 这里的i等价与上面公式中的2i和2i+1
    angle_rates = 1 / np.power(10000, (2*(i // 2))/ np.float32(d_model))
    return pos * angle_rates

In [4]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                           np.arange(d_model)[np.newaxis,:],
                           d_model)
    # 第2i项使用sin
    sines = np.sin(angle_rads[:, 0::2])
    # 第2i+1项使用cos
    cones = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.concatenate([sines, cones], axis=-1)
    pos_encoding = pos_encoding[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

In [8]:
pos_encoding = positional_encoding(10, 20)
print(pos_encoding.shape)

(1, 10, 20)


In [9]:
pos_encoding

<tf.Tensor: id=3, shape=(1, 10, 20), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  1.0000000e+00,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00],
        [ 8.4147096e-01,  3.8767424e-01,  1.5782665e-01,  6.3053876e-02,
          2.5116222e-02,  9.9998331e-03,  3.9810613e-03,  1.5848925e-03,
          6.3095731e-04,  2.5118864e-04,  5.4030228e-01,  9.2179644e-01,
          9.8746681e-01,  9.9801010e-01,  9.9968451e-01,  9.9994999e-01,
          9.9999207e-01,  9.9999875e-01,  9.9999982e-01,  9.9999994e-01],
        [ 9.0929741e-01,  7.1471345e-01,  3.1169716e-01,  1.2585682e-01,
          5.0216600e-02,  1.9998666e-02,  7.9620592e-03,  3.1697811e-03,
          1.2619144e-03,  5.0237728e-04, -4.1614684e-01,  6.994